In [1]:
# Import 
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
from geopy.distance import geodesic
from sklearn.preprocessing import StandardScaler

C:\Users\50193\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ----------------------------
# Step 1: Load and preprocess data
# ----------------------------
# Load the bike trip data
data = pd.read_csv('zhong.csv')

# Convert time columns to datetime format and extract features
data['start_time'] = pd.to_datetime(data['start_time'])
data['hour'] = data['start_time'].dt.hour

# Calculate total demand for each station and hour
demand_data = data.groupby(['end_station_name', 'hour']).size().rename('total_demand').reset_index()

# Normalize total demand
scaler = MinMaxScaler()
demand_data['total_demand'] = scaler.fit_transform(demand_data[['total_demand']])

# Add sine and cosine time features for periodicity
demand_data['hour_sin'] = np.sin(2 * np.pi * demand_data['hour'] / 24)
demand_data['hour_cos'] = np.cos(2 * np.pi * demand_data['hour'] / 24)

# Assign station IDs
demand_data['station_id'] = demand_data['end_station_name'].factorize()[0]


In [3]:
# ----------------------------
# Step 2: Create graph structure
# ----------------------------
# Load station coordinates for edges
station_coords = data[['end_station_name', 'end_lat', 'end_lng']].drop_duplicates()
station_coords = station_coords.dropna(subset=['end_lat', 'end_lng'])
station_coords = station_coords[
    (station_coords['end_lat'].apply(np.isfinite)) & (station_coords['end_lng'].apply(np.isfinite))
]
station_coords.columns = ['station_name', 'lat', 'lng']

# Build edge connections based on station proximity
edges = []
edge_weights = []
stations = station_coords.to_dict('records')

for i, station1 in enumerate(stations):
    for j, station2 in enumerate(stations):
        if i != j:
            distance = geodesic((station1['lat'], station1['lng']), (station2['lat'], station2['lng'])).kilometers
            if distance <= 1.0:  # Only consider stations within 1 km
                edges.append((station1['station_name'], station2['station_name']))
                edge_weights.append(1 / distance)  # Weight inversely proportional to distance

# Convert station names to indices for PyTorch Geometric
station_index_map = dict(zip(demand_data['end_station_name'], demand_data['station_id']))
edges = [(station_index_map[edge[0]], station_index_map[edge[1]]) for edge in edges]
edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
edge_attr = torch.tensor(edge_weights, dtype=torch.float)


In [4]:
# ----------------------------
# Step 3: Prepare node features and labels
# ----------------------------

# Get the number of stations and the embedding dimension
num_stations = demand_data['station_id'].nunique()
station_embedding_dim = 8  # Embedding dimension

# Convert station IDs to tensors
station_ids = torch.tensor(demand_data['station_id'].values, dtype=torch.long)  # [num_samples]

# Convert time features to tensors
time_features = torch.tensor(demand_data[['hour_sin', 'hour_cos']].values, dtype=torch.float)  # [num_samples, 2]

# Prepare labels
labels = demand_data['total_demand'].values
y = torch.tensor(labels, dtype=torch.float).unsqueeze(1)  # [num_samples, 1]


In [5]:
# ----------------------------
# Step 4: Split data into train/test
# ----------------------------
train_mask, test_mask = train_test_split(range(time_features.size(0)), test_size=0.2, random_state=42)
train_mask = torch.tensor(train_mask, dtype=torch.long)
test_mask = torch.tensor(test_mask, dtype=torch.long)



In [6]:
# ----------------------------
# Step 5: Define GNN model
# ----------------------------
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_stations, station_embedding_dim):
        super(GCN, self).__init__()
        # Define the embedding layer
        self.station_embedding = nn.Embedding(num_stations, station_embedding_dim)  # Embedding layer
        # Define graph convolutional layers
        self.conv1 = GCNConv(input_dim + station_embedding_dim, hidden_dim)  # Input includes embedding dimension
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, time_features, station_ids, edge_index):
        # Obtain station embeddings
        station_embedded = self.station_embedding(station_ids)  # [num_samples, station_embedding_dim]
        # Combine time features and station embeddings
        x = torch.cat([time_features, station_embedded], dim=1)  # [num_samples, input_dim + station_embedding_dim]
        # Apply graph convolution operations
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x


# Initialize the model
# Get input dimension
input_dim = time_features.size(1)  # Time feature dimensions (hour_sin, hour_cos)

# Get embedding layer parameters
num_stations = demand_data['station_id'].nunique()  # Total number of stations
station_embedding_dim = 8  # Embedding dimension

# Define hidden layer dimension and output dimension
hidden_dim = 16
output_dim = 1

# Initialize the model
model = GCN(input_dim, hidden_dim, output_dim, num_stations, station_embedding_dim)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


In [7]:
# ----------------------------
# Step 6: Train the model
# ----------------------------

# Define the model
input_dim = 2  # Time feature dimensions (hour_sin, hour_cos)
hidden_dim = 16
output_dim = 1
model = GCN(input_dim, hidden_dim, output_dim, num_stations, station_embedding_dim)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0013)

# Start training
for epoch in range(3000):
    model.train()
    optimizer.zero_grad()
    # Pass time_features and station_ids to the model
    out = model(time_features, station_ids, edge_index)
    loss = criterion(out[train_mask], y[train_mask])
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch + 1}, Loss: {loss.item():.4f}')


Epoch 10, Loss: 0.1268
Epoch 20, Loss: 0.0432
Epoch 30, Loss: 0.0272
Epoch 40, Loss: 0.0156
Epoch 50, Loss: 0.0115
Epoch 60, Loss: 0.0096
Epoch 70, Loss: 0.0083
Epoch 80, Loss: 0.0076
Epoch 90, Loss: 0.0070
Epoch 100, Loss: 0.0066
Epoch 110, Loss: 0.0063
Epoch 120, Loss: 0.0061
Epoch 130, Loss: 0.0059
Epoch 140, Loss: 0.0057
Epoch 150, Loss: 0.0056
Epoch 160, Loss: 0.0055
Epoch 170, Loss: 0.0055
Epoch 180, Loss: 0.0054
Epoch 190, Loss: 0.0054
Epoch 200, Loss: 0.0053
Epoch 210, Loss: 0.0053
Epoch 220, Loss: 0.0053
Epoch 230, Loss: 0.0052
Epoch 240, Loss: 0.0052
Epoch 250, Loss: 0.0051
Epoch 260, Loss: 0.0051
Epoch 270, Loss: 0.0050
Epoch 280, Loss: 0.0050
Epoch 290, Loss: 0.0050
Epoch 300, Loss: 0.0049
Epoch 310, Loss: 0.0049
Epoch 320, Loss: 0.0049
Epoch 330, Loss: 0.0049
Epoch 340, Loss: 0.0049
Epoch 350, Loss: 0.0049
Epoch 360, Loss: 0.0049
Epoch 370, Loss: 0.0049
Epoch 380, Loss: 0.0049
Epoch 390, Loss: 0.0048
Epoch 400, Loss: 0.0048
Epoch 410, Loss: 0.0048
Epoch 420, Loss: 0.0048
E

In [8]:
# ----------------------------
# Step 7: Evaluate the model
# ----------------------------

model.eval()
with torch.no_grad():
    predictions = model(time_features, station_ids, edge_index)
    test_loss = criterion(predictions[test_mask], y[test_mask])
    print(f'Test Loss: {test_loss.item():.4f}')

# Denormalize predictions for comparison
total_demand_scaled = np.zeros((predictions.shape[0], 1))  # Only total_demand
total_demand_scaled[:, 0] = predictions.detach().numpy().flatten()
predictions_denormalized = scaler.inverse_transform(total_demand_scaled)[:, 0]

actual_scaled = np.zeros((y.shape[0], 1))
actual_scaled[:, 0] = y.numpy().flatten()
actual_denormalized = scaler.inverse_transform(actual_scaled)[:, 0]

# Print sample predictions vs actual values
for i in range(10):
    print(f'Predicted: {max(1, predictions_denormalized[test_mask[i]]):.2f}, Actual: {actual_denormalized[test_mask[i]]:.2f}')


Test Loss: 0.0021
Predicted: 7.28, Actual: 10.00
Predicted: 3.37, Actual: 2.00
Predicted: 13.09, Actual: 12.00
Predicted: 2.25, Actual: 1.00
Predicted: 2.35, Actual: 1.00
Predicted: 37.18, Actual: 41.00
Predicted: 15.95, Actual: 22.00
Predicted: 18.02, Actual: 16.00
Predicted: 1.87, Actual: 1.00
Predicted: 2.69, Actual: 1.00


In [ ]:
# ----------------------------
# Step 8: Predict future demand
# ----------------------------

# Generate future time features for the next 24 hours
future_hours = np.arange(24)  # Hours from 0 to 23
future_time_features = pd.DataFrame({
    'hour': np.tile(future_hours, num_stations),
    'hour_sin': np.tile(np.sin(2 * np.pi * future_hours / 24), num_stations),
    'hour_cos': np.tile(np.cos(2 * np.pi * future_hours / 24), num_stations),
    'station_id': np.repeat(np.arange(num_stations), 24)
})

# Ensure station_id is mapped correctly and consistently
future_time_features['station_name'] = future_time_features['station_id'].map(
    {v: k for k, v in station_index_map.items()}
)

# Drop duplicate entries (if any, just in case)
future_time_features = future_time_features.drop_duplicates(subset=['station_name', 'hour'])

# Convert to tensors for prediction
future_station_ids = torch.tensor(future_time_features['station_id'].values, dtype=torch.long)
future_time_tensor = torch.tensor(future_time_features[['hour_sin', 'hour_cos']].values, dtype=torch.float)

# Predict demand for each station and hour
model.eval()
with torch.no_grad():
    future_predictions = model(future_time_tensor, future_station_ids, edge_index)

# Denormalize predictions
future_predictions_scaled = future_predictions.numpy().flatten()
future_predictions_denormalized = scaler.inverse_transform(future_predictions_scaled.reshape(-1, 1)).flatten()

# Add predictions back to DataFrame
future_time_features['predicted_demand'] = future_predictions_denormalized

# ----------------------------
# Step 9: Find top 10 stations and times
# ----------------------------

# Sort by predicted demand in descending order
top_20 = future_time_features.sort_values(by='predicted_demand', ascending=False).head(20)

# Ensure station_name and hour are unique in the results
top_20 = top_20.drop_duplicates(subset=['station_name', 'hour'])

# Map station_id back to station names (already done)
id_to_station = {v: k for k, v in station_index_map.items()}
top_20['station_name'] = top_20['station_id'].map(id_to_station)

# Print the top 20 results
print("Top 20 stations with the highest predicted demand:")
print(top_20[['station_name', 'hour']])


Top 20 stations with the highest predicted demand:
                    station_name  hour
2871           Clark St & Elm St    15
2872           Clark St & Elm St    16
2870           Clark St & Elm St    14
2869           Clark St & Elm St    13
4479       Dearborn St & Erie St    15
4480       Dearborn St & Erie St    16
2873           Clark St & Elm St    17
4478       Dearborn St & Erie St    14
2868           Clark St & Elm St    12
4477       Dearborn St & Erie St    13
4481       Dearborn St & Erie St    17
4599   Desplaines St & Kinzie St    15
13095  Wabash Ave & Roosevelt Rd    15
2867           Clark St & Elm St    11
4476       Dearborn St & Erie St    12
2874           Clark St & Elm St    18
4600   Desplaines St & Kinzie St    16
4598   Desplaines St & Kinzie St    14
13096  Wabash Ave & Roosevelt Rd    16
13094  Wabash Ave & Roosevelt Rd    14
